# Exercise 5.1


In [57]:
import numpy as np
import matplotlib.pyplot as plt

**Simple Network**

We continue with the dataset first encountered in the previous exercise. Please refer to the discussion there for an introduction to the data and the learning objective.

Here, we manually implement a simple network architecture

In [ ]:
# The code snippet below is responsible for downloading the dataset
# - for example when running via Google Colab.
#
# You can also directly download the file using the link if you work
# with a local setup (in that case, ignore the !wget)

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv

In [ ]:
# Before working with the data, 
# we download and prepare all features

# load all examples from the file
data = np.genfromtxt('winequality-white.csv',delimiter=";",skip_header=1)

print("data:", data.shape)

# Prepare for proper training
np.random.shuffle(data) # randomly sort examples

# take the first 3000 examples for training
# (remember array slicing from last week)
X_train = data[:3000,:11] # all features except last column
y_train = data[:3000,11]  # quality column

# and the remaining examples for testing
X_test = data[3000:,:11] # all features except last column
y_test = data[3000:,11] # quality column

print("First example:")
print("Features:", X_train[0])
print("Quality:", y_train[0])

# Problems

The goal is to implement the training of a neural network with one input layer, one hidden layer, and one output layer using gradient descent. We first (below) define the matrices and initialise with random values. We need W, b, W' and b'. The shapes will be:
  * W: (number of hidden nodes, number of inputs) named `W`
  * b: (number of hidden nodes) named `b`
  * W': (number of hidden nodes) named `Wp`
  * b': (one) named `bp`

Your tasks are:     
   * Implement a forward pass of the network as `dnn` (see below)
   * Implement a function that uses one data point to update the weights using gradient descent. You can follow the `update_weights` skeleton below
   * Now you can use the code below (training loop and evaluation) to train the network for multiple data points and even over several epochs. Try to find a set of hyperparameters (number of nodes in the hidden layer, learning rate, number of training epochs) that gives stable results. What is the best result (as measured by the loss on the training sample) you can get?

In [ ]:
# Initialise weights with suitable random distributions
hidden_nodes = 50 # number of nodes in the hidden layer
n_inputs = 11 # input features in the dataset

W = np.random.randn(hidden_nodes,11)*np.sqrt(2./n_inputs)
print(f"W shape: {W.shape}")
b = np.random.randn(hidden_nodes)*np.sqrt(2./n_inputs)
print(f"b shape: {b.shape}")
Wp = np.random.randn(hidden_nodes)*np.sqrt(2./hidden_nodes)
print(f"Wp shape: {Wp.shape}")
bp = np.random.randn((1))
print(f"bp shape: {bp.shape}")

x = X_train[0]
print(f"X shape: {X_train.shape}")
print(f"x shape: {x.shape}")

test = W @ x + b
print(test.shape)
test2 = Wp @ test + bp
print(test2.shape)

In [61]:
# You can use this implementation of the ReLu activation function
def relu(x):
    return np.maximum(x, 0)

In [62]:
def dnn(x,W,b,Wp,bp):
    # Calculate and return network output of forward pass
    output = Wp @ relu(W @ x + b) + bp  
    return output

In [63]:
def update_weights(x,y, W, b, Wp, bp):
    
    learning_rate = 0.0001

    # Calculate the network output (use the function dnn defined above)
    y_nn_train = dnn(x,W,b,Wp,bp)
  
    # Derive the gradient for each of W,b,Wp,bp by taking the partial
    # derivative of the loss function with respect to the variable
    
    # W gradient
    Grad_bp = 2*(y_nn_train - y)

    # b gradient
    Grad_b = np.ones(b.shape[0])
    for i_b in range(b.shape[0]):
      Grad_b = 2*(y_nn_train - y) * Wp[i_b] * np.heaviside((sum(W[i_b,:]*x) + b[i_b]),0)

    # Wp gradient
    Grad_Wp = np.ones(Wp.shape[0])
    for i_Wp in range(Wp.shape[0]):
      Grad_Wp = 2*(y_nn_train - y) * relu(sum(W[i_Wp,:]*x) + b[i_Wp])

    # bp gradient
    Grad_W = np.ones((W.shape[0],W.shape[1]))
    for m_W in range(W.shape[0]):
        for i_W in range(W.shape[1]):
            Grad_W[m_W,i_W] = 2*(y_nn_train - y) * Wp[m_W] * np.heaviside((sum(W[m_W,:]*x) + b[m_W]),0) * x[i_W]
    
    # You might need these numpy functions:
    # np.dot, np.outer, np.heaviside 
    
    # Update the weights/bias following the rule:  weight_new = weight_old - learning_rate * gradient 
    W_new = W - learning_rate * Grad_W
    b_new = b - learning_rate * Grad_b
    Wp_new = Wp - learning_rate * Grad_Wp
    bp_new = bp - learning_rate * Grad_bp

    return W_new, b_new, Wp_new, bp_new # return the new weights

# Training loop and evaluation below

In [ ]:
# The code below implements the training.
# If you correctly implement  dnn and update_weights above, 
# you should not need to change anything below. 
# (apart from increasing the number of epochs)

train_losses = []
test_losses = []

# How many epochs to train
# This will just train for one epoch
# You will want a higher number once everything works
n_epochs = 500 

# Loop over the epochs
for ep in range(n_epochs):
        
    # Each epoch is a complete over the training data
    for i in range(X_train.shape[0]):
        
        # pick one example
        x = X_train[i]
        y = y_train[i]

        # use it to update the weights
        W,b,Wp,bp = update_weights(x,y,W,b,Wp,bp)
    
    # Calculate predictions for the full training and testing sample
    y_pred_train = [dnn(x,W,b,Wp,bp)[0] for x in X_train]
    y_pred = [dnn(x,W,b,Wp,bp)[0] for x in X_test]

    # Calculate aver loss / example over the epoch
    train_loss = sum((y_pred_train-y_train)**2) / y_train.shape[0]
    test_loss = sum((y_pred-y_test)**2) / y_test.shape[0] 
    
    # print some information
    print("Epoch:",ep, "Train Loss:", train_loss, "Test Loss:", test_loss)
    
    # and store the losses for later use
    train_losses.append(train_loss)
    test_losses.append(test_loss)
    
    
# After the training:
    
# Prepare scatter plot
y_pred = [dnn(x,W,b,Wp,bp)[0] for x in X_test]

print("Best loss:", min(test_losses), "Final loss:", test_losses[-1])

print("Correlation coefficient:", np.corrcoef(y_pred,y_test)[0,1])
plt.scatter(y_pred_train,y_train)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

# Prepare and loss over time
plt.plot(train_losses,label="train")
plt.plot(test_losses,label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.show()


# Results

First, it is worth to mention that my code, being not very much optimized (I have very inefficiente loop in my code and I'm planning to improve it later), is slow and consequently, the smaller learning I have used is 0.0001, largest number of epochs was 30, and largest number of hidden nodes was 100. 

Several simulations, combining the following hyper-parameters values, have been ran:

learning rate: 0.01, 0.001, 0.0001
Hidden nodes: 100, 50, 15
epochs: 30

The first observation concernes the minimal loss obtain which is equal to 0.7612. If one could think that the deeper the network is and the smaller the learning rate better will be result, it isn't the case here since this best result was obtain with a learning rate of 0.01 and 100 hidden neurons in only one epoch...

Changing the hyper-parameter didn't give big difference of optimization since the worst result was a loss converging at 0.90 in 24 epochs and 15 hidden neurons, which was the largest number of epochs needed to converge.

Overall, there isn't, in my opinion, any pattern coming out of the different trial, small and large networks were converging fast and to a similar loss value, as well as the learning rate.

One can conclude that finding the optimial parameter is trial and error process, with in addition randomness from weights initialization, dataset split randomisation, and convergence to local minima.


# Hint 1

We want a network with one hidden layer. As activiation in the hidden layer $\sigma$ we apply element-wise ReLu, while no activation is used for the output layer. The forward pass of the network then reads:
$$\hat{y}=\mathbf{W}^{\prime} \sigma(\mathbf{W} \vec{x}+\vec{b})+b^{\prime}$$

# Hint 2

For the regression problem the objective function is the mean squared error between the prediction and the true label $y$: 
$$
L=(\hat{y}-y)^{2}
$$

Taking the partial derivatives - and diligently the applying chain rule - with respect to the different objects yields:

$$
\begin{aligned}
\frac{\partial L}{\partial b^{\prime}} &=2(\hat{y}-y) \\
\frac{\partial L}{\partial b_{k}} &=2(\hat{y}-y) \mathbf{W}_{k}^{\prime} \theta\left(\sum_{i} \mathbf{W}_{i k} x_{i}+b_{k}\right) \\
\frac{\partial L}{\partial \mathbf{W}_{k}^{\prime}} &=2(\hat{y}-y) \sigma\left(\sum_{i} \mathbf{W}_{i k} x_{i}+b_{k}\right) \\
\frac{\partial L}{\partial \mathbf{W}_{k m}} &=2(\hat{y}-y) \mathbf{W}_{m}^{\prime} \theta\left(\sum_{i} \mathbf{W}_{i m} x_{i}+b_{m}\right) x_{k}
\end{aligned}
$$

Here, $\Theta$ denotes the Heaviside step function.